In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
from pathlib import Path
import xtrack as xt
import xpart as xp
import xcoll as xc
import xobjects as xo
from matplotlib import colormaps

In [3]:
import pyarrow as pa
import pyarrow.parquet as pq
import numpy as np
import json

In [4]:
with open('../20251118/initial_conditions_uniform_ring_6d.json', 'r') as f:
    initial_conditions = json.load(f)

In [5]:
from pathlib import Path

file_path = Path("../20251118/initial_conditions_uniform_ring_6d.json")
size_bytes = file_path.stat().st_size
print(f"Size: {size_bytes / (1024**3):.2f} GB")

Size: 0.00 GB


In [6]:
job_id = 0

#Information
num_particles = 10
num_turns = 1_000_000
nemitt_x = 2e-6
nemitt_y = 2e-6
sigma_z = 00.224

rescale_factors = {'x_norm': 30,
                   'px_norm': 30,
                   'y_norm': 30,
                   'py_norm': 30,
                   'zeta': 3*sigma_z,
                   'delta': 5e-3}

In [58]:
line = xt.Line.from_json('../../injection_lines/sps_q20_inj.json')
tt = line.get_table()
tw = line.twiss()

context = xo.ContextCpu()
line.vars['qph_setvalue'] = 0.5
line.vars['qpv_setvalue'] = 0.5
line.vars['qh_setvalue'] = line.vars['qx0']._value + 0.05
line.vars['qv_setvalue'] = line.vars['qy0']._value + 0.05

cavity_elements, cavity_names = line.get_elements_of_type(xt.Cavity)

for name in cavity_names:
    line[name].frequency = 200e6
    line[name].lag = 180
line['acl.31735'].voltage = 0 #setting 800 cav to 0V
line['actcse.31632'].voltage = 3.0e6

Loading line from dict:   0%|          | 0/4037 [00:00<?, ?it/s]

Done loading line from dict.           


In [18]:
env = line.env
monitor = xt.ParticlesMonitor(num_particles=num_particles, start_at_turn=0, stop_at_turn=num_turns)
env.elements['monitor_start'] = monitor
line.insert([env.place('monitor_start', at=0)])

Slicing line:   0%|          | 0/3848 [00:00<?, ?it/s]

In [59]:
with open('../20251118/initial_conditions_uniform_ring_6d.json', 'r') as f:
    initial_conditions = json.load(f)

part = line.build_particles(
    x_norm=np.array(initial_conditions['x_norm'][job_id])*rescale_factors['x_norm'], px_norm=np.array(initial_conditions['px_norm'][job_id])*rescale_factors['px_norm'],
    y_norm=np.array(initial_conditions['y_norm'][job_id])*rescale_factors['y_norm'], py_norm=np.array(initial_conditions['py_norm'][job_id])*rescale_factors['py_norm'],
    zeta=np.array(initial_conditions['zeta'][job_id])*rescale_factors['zeta'],
    delta=np.array(initial_conditions['delta'][job_id])*rescale_factors['delta'],
    nemitt_x=2e-6, nemitt_y=2e-6, # normalized emittances
    )

In [60]:
line.discard_tracker()
line.build_tracker(_context=xo.ContextCpu(omp_num_threads='auto'))
part_dict = {pid + job_id*num_particles: {'x': [part.x[part.particle_id==pid][0].copy()], 'px': [part.px[part.particle_id==pid][0].copy()],
                                          'y': [part.y[part.particle_id==pid][0].copy()], 'py': [part.py[part.particle_id==pid][0].copy()],
                                          'zeta' : [part.zeta[part.particle_id==pid][0].copy()], 'delta' : [part.delta[part.particle_id==pid][0].copy()],
                                            'state': [part.state[part.particle_id==pid][0].copy()], 'at_turn': [0], 'particle_id': [pid], 'at_element': [0]} for pid in part.particle_id}

for nn_saving in range(int(num_turns/1000)):
    line.track(part, num_turns=1000, time=True, with_progress=True)
    for pid in part.particle_id:
        dict_id = job_id * num_particles + pid
        for key in part_dict[dict_id].keys():
            part_dict[dict_id][key].append(part.__getattribute__(key)[part.particle_id==pid][0].copy())

Compiling ContextCpu kernels...


ld: warning: duplicate -rpath '/Users/lisepauwels/miniforge3/envs/xcoll/lib' ignored
ld: warning: duplicate -rpath '/Users/lisepauwels/miniforge3/envs/xcoll/lib' ignored


Done compiling ContextCpu kernels.


Tracking:   0%|          | 0/1000 [00:00<?, ?it/s]

Tracking:   0%|          | 0/1000 [00:00<?, ?it/s]

...

Tracking:   0%|          | 0/1000 [00:00<?, ?it/s]

Tracking:   0%|          | 0/1000 [00:00<?, ?it/s]

Tracking:   0%|          | 0/1000 [00:00<?, ?it/s]

Tracking:   0%|          | 0/1000 [00:00<?, ?it/s]

Tracking:   0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
part_dict = {
    job_id * pid : 
}

In [57]:
float(part.x[part.particle_id==0][0])

0.03047229896961667

In [65]:
len(part_dict[0].keys())

10

In [74]:
columns = {}
array_length = 1001  # fixed size of each array

for key in part_dict[0]:  # iterate over keys
    # concatenate arrays for all pids for this key
    concatenated = np.concatenate([part_dict[pid][key] for pid in part_dict])
    # create FixedSizeListArray
    columns[key] = pa.FixedSizeListArray.from_arrays(concatenated, array_length)

# Optional: create a 'pid' column
pid_column = pa.array(list(part_dict.keys()))

# Build table
table = pa.table({**{"pid": pid_column}, **columns})
pq.write_table(table, "part_dict.parquet", compression="gzip")

In [35]:
file_path = Path("output.parquet")
size_bytes = file_path.stat().st_size
print(f"Size: {size_bytes / (1024**3):.2f} GB")

Size: 0.49 GB


In [37]:
file_path = Path("output2.parquet")
size_bytes = file_path.stat().st_size
print(f"Size: {size_bytes / (1024**3):.2f} GB")

Size: 0.43 GB


In [38]:
0.43*1000

430.0

In [39]:
dict_monitor={'x' : monitor.x.tolist(), 'y' : monitor.y.tolist(), 'px' : monitor.px.tolist(), 'py' : monitor.py.tolist(), 'zeta' : monitor.zeta.tolist(), 'delta' : monitor.delta.tolist(), 'state': monitor.state.tolist()}
with open('monitor_data.json', 'w') as f:
    json.dump(dict_monitor, f)

In [40]:
file_path = Path("monitor_data.json")
size_bytes = file_path.stat().st_size
print(f"Size: {size_bytes / (1024**3):.2f} GB")

Size: 1.30 GB


In [75]:
file_path = Path("part_dict.parquet")
size_bytes = file_path.stat().st_size
print(f"Size: {size_bytes / (1024**3):.2f} GB")

Size: 0.00 GB


In [76]:
table_loaded = pq.read_table("part_dict.parquet")

In [77]:
table_loaded

pyarrow.Table
pid: int64
x: fixed_size_list<element: double>[1001]
  child 0, element: double
px: fixed_size_list<element: double>[1001]
  child 0, element: double
y: fixed_size_list<element: double>[1001]
  child 0, element: double
py: fixed_size_list<element: double>[1001]
  child 0, element: double
zeta: fixed_size_list<element: double>[1001]
  child 0, element: double
delta: fixed_size_list<element: double>[1001]
  child 0, element: double
state: fixed_size_list<element: int64>[1001]
  child 0, element: int64
at_turn: fixed_size_list<element: int64>[1001]
  child 0, element: int64
particle_id: fixed_size_list<element: int64>[1001]
  child 0, element: int64
at_element: fixed_size_list<element: int64>[1001]
  child 0, element: int64
----
pid: [[0,1,2,3,4,5,6,7,8,9]]
x: [[[-0.03227699624082492,-0.0007086907696425609,0.024974230241852635,0.03743616813101566,0.035254027466026426,...,0.037403348880833125,0.034516135533586394,0.021623597336094424,0.00526805084150221,-0.010020949883943739]